# Runtime notebook for Multi Lingual support of Generative AI Quality metrics for IBM watsonx.governance

This notebook demonstrates the Generative AI Quality monitor's runtime metric results for a summarization prompt task type in Japanese. You can also view the multi-lingual design time notebook [here](https://github.com/IBM/watson-openscale-samples/blob/main/WatsonX.Governance/Cloud/GenAI/samples/Multi%20Lingual%20Support%20Notebook.ipynb)

The metrics computed which support multiple languages across the various task types are :
- Summarization:
    - Rouge Score
    - Cosine Similarity
	- Jaccard Similarity
    - Normalized Precision
    - Normalized Recall
    - Normalized F1 Score
	- Sari
	- Meteor
    - HAP Score
    - PII
- Generation:
    - Rouge Score
    - Normalized Precision
    - Normalized Recall
    - Normalized F1 Score
	- Meteor
    - HAP Score
    - PII
- Extraction:
    - Exact Match
    - Rouge Score
- Question Answering(QA):
    - Exact Match
    - Rouge Score
    - HAP Score
    - PII
- Retrieval Augmented Generation(RAG)
    - Rouge Score
    - Exact Match
    - HAP Score
    - PII

**Note** : 
- The below given example is specific to the Japanese language.
- The current preferred method for calculating these metrics is when the prompt instruction, input data, and generated text are all in the same language. In other situations, the values will still be generated, but they may not be accurate.

The following is a list of supported languages together with the corresponding language code:
- English : en
- Japanese : ja
- German : de
- French : fr
- Spanish : es
- Arabic : ar
- Italian : it
- Portugese : pt
- Korean : ko
- Danish : da

## Contents

- [Step 1 - Setup](#setup)
- [Step 2 - Create the Japanese summarization prompt template](#prompt)
- [Step 3 - Trigger Prompt Setup](#ptatsetup)
- [Step 4 - Run Risk evaluations for prompt template asset subscription](#evaluate)
- [Step 5 - Display the Model Risk metrics](#mrmmetric)
- [Step 6 - Display the Generative AI Quality metrics](#genaimetrics)

## Step 1 - Setup <a name="setup"></a>

In [1]:
!pip install -U ibm_watson_openscale | tail -n 1
!pip install --upgrade ibm-watsonx-ai | tail -n 1

Note: you may need to restart the kernel to use updated packages.

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Provision services and configure credentials

If you have not already, provision an instance of IBM watsonx.governance using the [watsonx.governance link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watsonxgovernance).

In [ ]:
use_cpd = False
IAM_URL="https://iam.cloud.ibm.com"
DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
CLOUD_API_KEY = "<EDIT THIS>" # YOUR_CLOUD_API_KEY


WML_CREDENTIALS = {
	"url": "https://us-south.ml.cloud.ibm.com",
	"apikey": CLOUD_API_KEY
}

Uncomment the code and run the below cell only if you are running your notebook on a CPD cluster.

In [3]:
# use_cpd = True
# WOS_CREDENTIALS = {
#     "url": "xxxxx",
#     "username": "xxxxx",
#     "password": "xxxxx"
# }

# WML_CREDENTIALS = {
# 	"url": "<EDIT THIS>",
# 	"username": "<EDIT THIS>",
# 	"password" : "<EDIT THIS>",
# 	"instance_id": "<EDIT THIS>",
# 	"apikey": "<EDIT THIS>",
# 	"version" : "5.2" #If your env is CP4D 5.x.x then specify "5.x.x" instead of "5.2"
# }

## Read project id from user

In order to set up a development type subscription, the PTA must be within the project. Please supply the project ID where the PTA needs to be created.

In [4]:
project_id = "<EDIT THIS>" # YOUR_PROJECT_ID

## Step 2 - Create Prompt template <a name="prompt"></a>

Create a prompt template for summarization task in Japanese

In [5]:
if not use_cpd:
    credentials={
        "apikey": CLOUD_API_KEY,
        "url": "https://us-south.ml.cloud.ibm.com"
    }
else:
	credentials={
		"apikey": WML_CREDENTIALS["apikey"],
		"url": WML_CREDENTIALS["url"],
		"instance_id": "openshift",
		"username": WML_CREDENTIALS["username"]
	}

In [ ]:
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

prompt_mgr = PromptTemplateManager(
	credentials = credentials,
	project_id = project_id
)

prompt_template = PromptTemplate(name="MLS_Car_Insurance_Summarization_Ja_Demo",
                                 model_id=ModelTypes.GRANITE_13B_INSTRUCT_V2,   # For CPD you can use 'ModelTypes.FLAN_UL2'
                                 task_ids=["summarization"],
                                 instruction="以下の保険請求を最大 3 文で要約してください",
                                 input_text="{input_text}",
                                 input_variables=['input_text'],
                                 examples = [
										[
                                            "２０２３年１１月１日午前１１時、私の車両「日産　マキシマ　１９９８年式」は、ニューヨークで重大事故に遭いました。私の運転手は制限速度を守って走行していましたが、前の車が急停止したために前方から衝突しました。激しい衝撃により双方の車が大破し、私の運転手はすぐに救急に連絡をして、その場で応急処置を受けました。私の車両は前のバンパー、ボンネット、フロントガラスに損傷を受けました。私の運転手は重傷を負い、現在治療を受けています。私はあなたの代理人にただちに連絡し、事故の詳細と警察の報告書、医療診察書を添えて請求書を提出しました。また目撃者の供述書と破損した車両の写真も提出します。",
										"私の車両はニューヨークで重大事故に遭いました。私の運転手は重傷を負い、現在治療を受けています。"
										]
                                	]
                                )

stored_prompt_template = prompt_mgr.store_prompt(prompt_template)
project_pta_id = stored_prompt_template.prompt_id
project_pta_id

'c594bd74-1ece-486e-904e-0799ae54666e'

## Step 3 - Trigger Prompt setup <a name="ptatsetup"></a>

### Configure OpenScale client

In [7]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

if use_cpd:
    authenticator = CloudPakForDataAuthenticator(
            url=WOS_CREDENTIALS['url'],
            username=WOS_CREDENTIALS['username'],
            password=WOS_CREDENTIALS['password'],
            disable_ssl_verification=True
        )
    
    wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
    print(wos_client.version)
else:
	service_instance_id = None # Update this to refer to a particular service instance
	authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY, url = IAM_URL)
	wos_client = APIClient(authenticator=authenticator, service_url = SERVICE_URL, service_instance_id = service_instance_id)
	print(wos_client.version)

3.0.45


### Listing all the available datamarts

In [6]:
wos_client.data_marts.show()

AIOSFASTPATH-16F11000-6E49-483C-8D97-AD853415ED8D,None,True,active,2025-02-11 13:47:30.138000+00:00,16f11000-6e49-483c-8d97-ad853415ed8d


In [9]:
data_mart_id = "<EDIT THIS>" # YOUR_DATAMART_ID

### Openscale instance mapping with the project

When the authentication is on CPD then we need to add additional step of mapping the `project_id/space_id` to an OpenScale instance.

In [15]:
if use_cpd:
    wos_client.wos.add_instance_mapping(                
		service_instance_id=data_mart_id,
		project_id=project_id
	)

### Setup the prompt template asset in project for evaluation with supported monitor dimensions

The prompt template assets from project is only supported with `development` operational space ID. Running the below cell will create a development type subscription from the prompt template asset created within the project.

The available parameters that can be passed for `execute_prompt_setup` function are:

 * `prompt_template_asset_id` : Id of prompt template asset for which subscription needs to be created.
 * `label_column` :  The name of the column containing the ground truth or actual labels.
 * `project_id` : The GUID of the project.
 * `space_id` : The GUID of the space.
 * `deployment_id` : (optional) The GUID of the deployment.
 * `operational_space_id` : The rank of the environment in which the monitoring is happening. Accepted values are `development`, `pre_production`, `production`.
 * `problem_type` : (optional) The task type to monitor for the given prompt template asset.
 * `classification_type` : The classification type `binary`/`multiclass` applicable only for `classification` problem (task) type.
 * `input_data_type` : The input data type.
 * `supporting_monitors` : Monitor configuration for the subscription to be created.
 * `data_input_locale` : List containing locale codes of the input data provided.
 * `generated_output_locale` : List containing locale codes of the generated output data.
 * `background_mode` : When `True`, the promt setup operation will be executed in the background

**Note** : 
- The below given example is specific to the Japanese language.
- The support right now is only present when the prompt instruction, input data and the generated text all are in the same language
- The values for `data_input_locale` and `generated_output_locale` are immutable fields and currently support only a single langauge

In [10]:
label_column = "reference_summary"
operational_space_id = "development"
problem_type= "summarization"
input_data_type= "unstructured_text"
language_code = "ja" # Update this to calculate metrics for another language

monitors = {
    "generative_ai_quality": {
        "parameters": {
            "min_sample_size": 10,
            "metrics_configuration": {
                "rouge_score": {},
                "normalized_precision": {},
                "normalized_f1": {},
                "normalized_recall": {},
                "cosine_similarity": {},
                "jaccard_similarity": {},
                "sari": {},
                "meteor": {},
                "hap_score": {},
                "pii": {
                    "language_code" : language_code
                }
            }
        }
    }
}

response = wos_client.wos.execute_prompt_setup(
    prompt_template_asset_id = project_pta_id,
	project_id = project_id,
	label_column = label_column,
	operational_space_id = operational_space_id,
	problem_type = problem_type,
	input_data_type = input_data_type,
	supporting_monitors = monitors,
    data_input_locale = [language_code],
    generated_output_locale = [language_code],
	background_mode = False
)

result = response.result
result._to_dict()




 Waiting for end of adding prompt setup c594bd74-1ece-486e-904e-0799ae54666e 




running...
finished

---------------------------------------------------------------
 Successfully finished setting up prompt template subscription 
---------------------------------------------------------------




{'prompt_template_asset_id': 'c594bd74-1ece-486e-904e-0799ae54666e',
 'project_id': '80ef74dd-3845-4dfb-8cad-06de6c144988',
 'deployment_id': 'b1e1cd4f-c8c5-4ad8-ab2d-b2f44686fe6c',
 'service_provider_id': '01963a2f-7c9f-797d-a741-0b916dc2974d',
 'subscription_id': '01963a41-bdd1-74b8-9aa9-4d638964a180',
 'mrm_monitor_instance_id': '01963a41-f646-7bcb-9be4-12011305cee9',
 'start_time': '2025-04-15T16:21:45.043873Z',
 'end_time': '2025-04-15T16:22:12.726177Z',
 'status': {'state': 'FINISHED'}}

With the below cell, users can  read the  prompt setup task and check its status

In [11]:
response = wos_client.wos.get_prompt_setup(prompt_template_asset_id = project_pta_id,
                                                             project_id = project_id)

result = response.result
result_json = result._to_dict()

if result_json["status"]["state"] == "FINISHED":
    print("Finished prompt setup : The response is {}".format(result_json))
else:
    print("prompt setup failed The response is {}".format(result_json))

Finished prompt setup : The response is {'prompt_template_asset_id': 'c594bd74-1ece-486e-904e-0799ae54666e', 'project_id': '80ef74dd-3845-4dfb-8cad-06de6c144988', 'deployment_id': 'b1e1cd4f-c8c5-4ad8-ab2d-b2f44686fe6c', 'service_provider_id': '01963a2f-7c9f-797d-a741-0b916dc2974d', 'subscription_id': '01963a41-bdd1-74b8-9aa9-4d638964a180', 'mrm_monitor_instance_id': '01963a41-f646-7bcb-9be4-12011305cee9', 'start_time': '2025-04-15T16:21:45.043873Z', 'end_time': '2025-04-15T16:22:12.726177Z', 'status': {'state': 'FINISHED'}}


### Read subscription id from prompt setup

Once prompt setup status is finished, Read the subscription id from it.

In [12]:
dev_subscription_id = result_json["subscription_id"]
dev_subscription_id

'01963a41-bdd1-74b8-9aa9-4d638964a180'

# Risk evaluations for PTA subscription <a name="evaluate"></a>

### Evaluate the prompt template subscription

For the risk assessment of a development type subscription the user needs to have an evaluation dataset. The risk evaluation function takes the evaluation dataset path as a parameter for evaluation of the configured metric dimensions. If there is a discrepancy between the feature columns in the subscription and the column names in the uploading CSV, users has the option to supply a mapping JSON file to associate the CSV column names with the feature column names in the subscription.


**Note:** If you are running this notebook from Watson studio, you may first need to upload your test data to studio and run code snippet to download feedback data file from project to local directory

In [13]:
# Download summarisation data
filename = "llm_content_summarization_ja.csv"
!rm -fr "llm_content_summarization_ja.csv"
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/refs/heads/main/IBM%20Cloud/WML/assets/data/watsonx/Multi_Lingual_Support/llm_content_summarization_ja.csv"

--2025-04-15 16:22:19--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/refs/heads/main/IBM%20Cloud/WML/assets/data/watsonx/Multi_Lingual_Support/llm_content_summarization_ja.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26758 (26K) [text/plain]
Saving to: ‘llm_content_summarization_ja.csv’

llm_content_summari 100%[===================>]  26.13K  --.-KB/s    in 0.02s   

2025-04-15 16:22:19 (1.43 MB/s) - ‘llm_content_summarization_ja.csv’ saved [26758/26758]



Rename the generated predictions column to `generated_text` to avoid scoring since the prompt is in Japanese and we would need a Foundational model which supports Japanese

In [14]:
import pandas as pd

df = pd.read_csv(filename)
df = df.rename(columns={'generated_predictions': 'generated_text'})
new_summarization_filename = "new_summarization_data_ja.csv"
df.to_csv(new_summarization_filename, index=False)
test_data_path = new_summarization_filename

### Read the MRM monitor instance id

Evaluating the test data against the prompt template subscription requires the monitor instance ID of MRM.

In [15]:
monitor_definition_id = "mrm"
target_target_id = dev_subscription_id
result = wos_client.monitor_instances.list(data_mart_id=data_mart_id,
                                           monitor_definition_id=monitor_definition_id,
                                           target_target_id=target_target_id,
                                           project_id=project_id).result
result_json = result._to_dict()
mrm_monitor_id = result_json["monitor_instances"][0]["metadata"]["id"]
mrm_monitor_id

'01963a41-f646-7bcb-9be4-12011305cee9'

The following cell will assess the test data with the subscription of the prompt template asset and produce relevant measurements for the configured monitor.

In [16]:
test_data_set_name = "data"
content_type = "multipart/form-data"

response  = wos_client.monitor_instances.mrm.evaluate_risk(monitor_instance_id=mrm_monitor_id, 
                                                    test_data_set_name = test_data_set_name, 
                                                    test_data_path = test_data_path,
                                                    content_type = content_type,
                                                    body = None,
                                                    includes_model_output=True,
                                                    project_id = project_id,
                                                    background_mode = False)




 Waiting for risk evaluation of MRM monitor 01963a41-f646-7bcb-9be4-12011305cee9 




upload_in_progress.
running......
finished

---------------------------------------
 Successfully finished evaluating risk 
---------------------------------------




### Read the risk evaluation response

After initiating the risk evaluation, the evaluation results are now available for review

In [17]:
response  = wos_client.monitor_instances.mrm.get_risk_evaluation(mrm_monitor_id, project_id = project_id)
response.result.to_dict()

{'metadata': {'id': '21278e36-0be0-4cc2-bd51-136ecca1b2d2',
  'created_at': '2025-04-15T16:23:07.541Z',
  'created_by': 'internal-service'},
 'entity': {'triggered_by': 'user',
  'parameters': {'evaluation_start_time': '2025-04-15T16:22:48.425253Z',
   'evaluator_user_key': '3ab3c4dc-299c-4c84-9448-0f6f1118cf58',
   'facts': {'state': 'finished'},
   'is_auto_evaluated': False,
   'measurement_id': '01963a42-f0c9-786a-ac83-5d26a6e4c10b',
   'monitors_run_status': [{'monitor_id': 'generative_ai_quality',
     'status': {'state': 'finished'}},
    {'monitor_id': 'model_health', 'status': {'state': 'finished'}}],
   'project_id': '80ef74dd-3845-4dfb-8cad-06de6c144988',
   'prompt_template_asset_id': 'c594bd74-1ece-486e-904e-0799ae54666e',
   'user_iam_id': '1000331001',
   'wos_created_deployment_id': 'b1e1cd4f-c8c5-4ad8-ab2d-b2f44686fe6c',
   'publish_metrics': 'false',
   'evaluation_tests': ['drift_v2',
    'fairness',
    'generative_ai_quality',
    'model_health',
    'quality']},
 

# Display the Generative AI Quality metrics <a name="genaimetrics"></a>



Monitor instance ID of Generative ai quality metrics is required for reading its metrics.

In [18]:
monitor_definition_id = "generative_ai_quality"
result = wos_client.monitor_instances.list(data_mart_id = data_mart_id,
                                           monitor_definition_id = monitor_definition_id,
                                           target_target_id = target_target_id,
                                           project_id = project_id).result
result_json = result._to_dict()
genaiquality_monitor_id = result_json["monitor_instances"][0]["metadata"]["id"]
genaiquality_monitor_id

'01963a41-e774-720a-b301-8f6f8e2e91b4'

Displaying the GenAIQ monitor metrics generated through the risk evaluation.

In [19]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=genaiquality_monitor_id, project_id=project_id)

2025-04-15 16:24:09.323434+00:00,normalized_recall,01963a43-e16b-74a8-99b2-ac706540bedd,0.1613,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,rouge2,01963a43-e16b-74a8-99b2-ac706540bedd,0.0952,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,records_processed,01963a43-e16b-74a8-99b2-ac706540bedd,20.0,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,jaccard_similarity,01963a43-e16b-74a8-99b2-ac706540bedd,0.1475,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,meteor,01963a43-e16b-74a8-99b2-ac706540bedd,0.0858,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,rougelsum,01963a43-e16b-74a8-99b2-ac706540bedd,0.1609,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,hap_score,01963a43-e16b-74a8-99b2-ac706540bedd,0.0237,None,0.0,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,pii,01963a43-e16b-74a8-99b2-ac706540bedd,0.0,None,0.0,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,normalized_precision,01963a43-e16b-74a8-99b2-ac706540bedd,0.2353,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180
2025-04-15 16:24:09.323434+00:00,normalized_f1,01963a43-e16b-74a8-99b2-ac706540bedd,0.2299,0.8,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:min']",generative_ai_quality,01963a41-e774-720a-b301-8f6f8e2e91b4,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,subscription,01963a41-bdd1-74b8-9aa9-4d638964a180


Note: First 10 records were displayed.


## Display record level metrics for Generative AI Quality 

Read the dataset id for generative ai quality dataset

In [20]:
result = wos_client.data_sets.list(target_target_id = dev_subscription_id,
                                target_target_type = "subscription",
                                type = "gen_ai_quality_metrics").result

genaiq_dataset_id = result.data_sets[0].metadata.id
genaiq_dataset_id

'01963a41-ee28-7c4e-aa99-c3abd8286d68'

Displaying record level metrics for generative ai quality

In [21]:
wos_client.data_sets.show_records(data_set_id = genaiq_dataset_id)

0.3448275862068966,0.2083,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-0,0.30303030303030304,feedback,2025-04-15T16:22:50.573Z,0.31631205673758866,0.32,0.0478239506483078,0.0,0.47619047619047616,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.39999999999999997,None,0.32,35.81572523470902,0.33742073667851313,None,0.4
0.23214285714285715,0.1299,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-1,0.23636363636363636,feedback,2025-04-15T16:22:50.573Z,0.1491034775897361,0.2278,0.023666102439165115,0.0,0.5652173913043478,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.3291139240506329,None,0.2278,29.000679470356232,0.348701955972673,None,0.3291
0.3333333333333333,0.1667,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-10,0.21428571428571427,feedback,2025-04-15T16:22:50.573Z,0.30702966648240276,0.28,0.023666102439165115,0.0,0.391304347826087,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.36,None,0.28,35.2784149510787,0.23244398764175528,None,0.36
0.43902439024390244,0.4062,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-11,0.40476190476190477,feedback,2025-04-15T16:22:50.573Z,0.4367988970357837,0.4545,0.028214920312166214,0.0,0.72,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.5454545454545455,None,0.4545,41.64958160752452,0.4972134156783251,None,0.5455
0.6666666666666666,0.0952,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-12,0.22857142857142856,feedback,2025-04-15T16:22:50.573Z,0.3746698943661972,0.2727,0.0744711384177208,0.0,0.25,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.36363636363636365,None,0.2727,37.94577720532095,0.31019286302448973,None,0.3636
0.4782608695652174,0.1455,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-13,0.275,feedback,2025-04-15T16:22:50.573Z,0.3976201090497583,0.3158,0.10210409760475159,0.0,0.3235294117647059,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.3859649122807018,None,0.3158,40.662473714645216,0.3018702723868621,None,0.386
0.2,0.1017,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-14,0.1951219512195122,feedback,2025-04-15T16:22:50.573Z,0.16568241469816272,0.2295,0.0478239506483078,0.0,0.38095238095238093,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.2622950819672132,None,0.2295,30.100809546192014,0.2651015734590852,None,0.2623
0.3333333333333333,0.1667,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-15,0.21428571428571427,feedback,2025-04-15T16:22:50.573Z,0.30702966648240276,0.28,0.023666102439165115,0.0,0.391304347826087,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.36,None,0.28,35.2784149510787,0.23244398764175528,None,0.36
0.43902439024390244,0.4062,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-16,0.40476190476190477,feedback,2025-04-15T16:22:50.573Z,0.4367988970357837,0.4545,0.028214920312166214,0.0,0.72,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.5454545454545455,None,0.4545,41.64958160752452,0.4972134156783251,None,0.5455
0.6666666666666666,0.0952,[],MRM_3dbe67a5-d6d5-4482-a656-7860180483a3-17,0.22857142857142856,feedback,2025-04-15T16:22:50.573Z,0.3746698943661972,0.2727,0.0744711384177208,0.0,0.25,None,cd01eddf-be6b-4ef0-823e-3315c2cb5a49,0.36363636363636365,None,0.2727,37.94577720532095,0.31019286302448973,None,0.3636


Author: <a href="mailto:kshitij.g1@ibm.com">Kshitij Gopali</a>

Copyright © 2025 IBM.